In [1]:
import os
import networkx as nx
import numpy as np
import re

####### Global options #######
import configuration

pi = np.pi

vname_dict = {'V':1,'Er':2,'Ti':3,'Ce':4,'S':5,
			  'H':6,'He':7,'Li':8,'Be':9,'B':10,
			  'C':11,'N':12,'O':13,'F':14,'Ne':15,
			  'Na':16,'Mg':17,'Al':18,'Si':19,'P':20 ,
			  'Cl':21,'Ar':22,'K':23,'Ca':24,'Sc':24,
			  'Cr':26,'Mn':27,'Fe':28,'Co':29,'Ni':30}

metal_elements = ['Ac','Ag','Al','Am','Au','Ba','Be','Bi',
				  'Bk','Ca','Cd','Ce','Cf','Cm','Co','Cr',
				  'Cs','Cu','Dy','Er','Es','Eu','Fe','Fm',
				  'Ga','Gd','Hf','Hg','Ho','In','Ir',
				  'K','La','Li','Lr','Lu','Md','Mg','Mn',
				  'Mo','Na','Nb','Nd','Ni','No','Np','Os',
				  'Pa','Pb','Pd','Pm','Pr','Pt','Pu','Ra',
				  'Rb','Re','Rh','Ru','Sc','Sm','Sn','Sr',
				  'Ta','Tb','Tc','Th','Ti','Tl','Tm','U',
				  'V','W','Y','Yb','Zn','Zr']


####### Global options #######
IGNORE_ALL_ERRORS = configuration.IGNORE_ALL_ERRORS
#PRINT = configuration.PRINT
PRINT =True
CONNECTION_SITE_BOND_LENGTH = configuration.CONNECTION_SITE_BOND_LENGTH
WRITE_CHECK_FILES = configuration.WRITE_CHECK_FILES
WRITE_CIF = configuration.WRITE_CIF
ALL_NODE_COMBINATIONS = configuration.ALL_NODE_COMBINATIONS
USER_SPECIFIED_NODE_ASSIGNMENT = configuration.USER_SPECIFIED_NODE_ASSIGNMENT
COMBINATORIAL_EDGE_ASSIGNMENT = configuration.COMBINATORIAL_EDGE_ASSIGNMENT
#CHARGES = configuration.CHARGES
CHARGES = False
SCALING_ITERATIONS = configuration.SCALING_ITERATIONS
SYMMETRY_TOL = configuration.SYMMETRY_TOL
BOND_TOL = configuration.BOND_TOL
ORIENTATION_DEPENDENT_NODES = configuration.ORIENTATION_DEPENDENT_NODES
PLACE_EDGES_BETWEEN_CONNECTION_POINTS = configuration.PLACE_EDGES_BETWEEN_CONNECTION_POINTS
RECORD_CALLBACK = configuration.RECORD_CALLBACK
OUTPUT_SCALING_DATA = configuration.OUTPUT_SCALING_DATA
FIX_UC = configuration.FIX_UC
MIN_CELL_LENGTH = configuration.MIN_CELL_LENGTH
OPT_METHOD = configuration.OPT_METHOD
PRE_SCALE = configuration.PRE_SCALE
SINGLE_METAL_MOFS_ONLY = configuration.SINGLE_METAL_MOFS_ONLY
MOFS_ONLY = configuration.MOFS_ONLY
MERGE_CATENATED_NETS = configuration.MERGE_CATENATED_NETS
RUN_PARALLEL = configuration.RUN_PARALLEL
REMOVE_DUMMY_ATOMS = configuration.REMOVE_DUMMY_ATOMS

from ciftemplate2graph import ct2g
from vertex_edge_assign import vertex_assign, assign_node_vecs2edges
from cycle_cocyle import cycle_cocyle, Bstar_alpha
from bbcif_properties import cncalc, bbelems
from SBU_geometry import SBU_coords
from scale import scale
from scaled_embedding2coords import omega2coords
from place_bbs import scaled_node_and_edge_vectors, place_nodes, place_edges
from remove_net_charge import fix_charges
from remove_dummy_atoms import remove_Fr
from adjust_edges import adjust_edges
from write_cifs import write_check_cif, write_cif, bond_connected_components, distance_search_bond, fix_bond_sym, merge_catenated_cifs
from scale_animation import scaling_callback_animation, write_scaling_callback_animation, animate_objective_minimization
import itertools
from random import choice

In [ ]:
# ct2g(template.CIF)
# SG, start, unit_cell, set(cns), set(e_types), cifname, aL, bL, cL, alpha, beta, gamma, max_le, catenation

In [ ]:

#@TG template graph
#@start fractional coordinate
#@unit_cell Cartesian coordinate
#@TVT Template Vertex Table(the degree (number of connections) of node n in the main graph G, node type)
#@TET template ((l[0],l[1]))) edge connected node pair (only numeric part of the node identifier)
#@TNAME template name
#@catenation if G has seperated (connected) subgraph


In [ ]:
import matplotlib.pyplot as pl
G1 = nx.path_graph(4)
nx.add_path(G1, [10, 11, 12])
nx.add_path(G1, [9, 11, 13])
l = [len(c) for c in sorted(nx.connected_components(G1), key=len, reverse=True)]
print(l,sorted(nx.connected_components(G1)))
pl.figure()
nx.draw_networkx(G1)
pl.show()

In [35]:
template = 'cds.cif'
PLACE_EDGES_BETWEEN_CONNECTION_POINTS = True
PRINT=False
print()
print('=========================================================================================================')
print('template :',template)                                          
print('=========================================================================================================')
print()
	
cat_count = 0
for net in ct2g(template):

		cat_count += 1
		TG, start, unit_cell, TVT, TET, TNAME, a, b, c, ang_alpha, ang_beta, ang_gamma, max_le, catenation = net

		TVT = sorted(TVT, key=lambda x:x[0], reverse=True) # sort node with connected degree, the first one is the highest(full)-coordinated node
		TET = sorted(TET, reverse=True) #sort node_pair by the node_index
		#get node cif information from node dir
		node_cns = [(cncalc(node, 'nodes'), node) for node in os.listdir('nodes')]

		print('Number of vertices = ', len(TG.nodes()))
		print('Number of edges = ', len(TG.edges()))
		print()

		edge_counts = dict((data['type'],0) for e0,e1,data in TG.edges(data=True))
		for e0,e1,data in TG.edges(data=True):
			edge_counts[data['type']] += 1
		
		if PRINT:
	
			print('There are', len(TG.nodes()), 'vertices in the voltage graph:')
			print()
			v = 0
	
			for node in TG.nodes():
				v += 1
				print(v,':',node)
				node_dict = TG.nodes[node]
				print('type : ', node_dict['type'])
				print('cartesian coords : ', node_dict['ccoords'])
				print('fractional coords : ', node_dict['fcoords'])
				#print('degree : ', node_dict['cn'][0])
				print()
	
			print('There are', len(TG.edges()), 'edges in the voltage graph:')
			print()
	
			for edge in TG.edges(data=True,keys=True):
				edge_dict = edge[3]
				ind = edge[2]
				print(ind,':',edge[0],edge[1])
				print('length : ',edge_dict['length'])
				print('type : ',edge_dict['type'])
				print('label : ',edge_dict['label'])
				print('positive direction :',edge_dict['pd'])
				print('cartesian coords : ',edge_dict['ccoords'])
				print('fractional coords : ',edge_dict['fcoords'])
				print()
	
		vas = vertex_assign(TG, TVT, node_cns, unit_cell, USER_SPECIFIED_NODE_ASSIGNMENT, SYMMETRY_TOL, ALL_NODE_COMBINATIONS)
		CB,CO = cycle_cocyle(TG)

		for va in vas:
			if len(va) == 0:
				print('At least one vertex does not have a building block with the correct number of connection sites.')
				print('Moving to the next template...')
				print()
				continue
	
		if len(CB) != (len(TG.edges()) - len(TG.nodes()) + 1):
			print('The cycle basis is incorrect.')
			print('The number of cycles in the cycle basis does not equal the rank of the cycle space.')
			print('Moving to the next template...')
			continue
		
		num_edges = len(TG.edges())
		Bstar, alpha = Bstar_alpha(CB,CO,TG,num_edges)

		if PRINT:
			print('B* (top) and alpha (bottom) for the barycentric embedding are:')
			print()
			for i in Bstar:
				print(i)
			print()
			for i in alpha:
				print(i)
			print()
	
		num_vertices = len(TG.nodes())
	
		if COMBINATORIAL_EDGE_ASSIGNMENT:
			eas = list(itertools.product([e for e in os.listdir('edges')], repeat = len(TET)))
		else:
			edge_files = sorted([e for e in os.listdir('edges')])
			eas = []
			i = 0
			while len(eas) < len(TET):
				eas.append(edge_files[i])
				i += 1
				if i == len(edge_files):
					i = 0
			eas = [eas]
	
		g = 0

		for va in vas:
			#check if assigned node has metal element 
			node_elems = [bbelems(i[1], 'nodes') for i in va]
			metals = [[i for i in j if i in metal_elements] for j in node_elems]
			metals = list(set([i for j in metals for i in j]))
			#set node cif files as vertex assignment
			v_set0 = [('v' + str(vname_dict[re.sub('[0-9]','',i[0])]), i[1]) for i in va]
			v_set1 = sorted(list(set(v_set0)), key=lambda x: x[0])
			v_set = [v[0] + '-' + v[1] for v in v_set1]
	
			print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
			print('vertex assignment : ',v_set)
			print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
			print()

			if SINGLE_METAL_MOFS_ONLY and len(metals) != 1:
				print(v_set, 'contains no metals or multiple metal elements, no cif will be written')
				print()
				continue

			if MOFS_ONLY and len(metals) < 1:
				print(v_set, 'contains no metals, no cif will be written')
				print()
				continue
			
			# add cifname to TG.nodes
			for v in va:
				for n in TG.nodes(data=True):
					if v[0] == n[0]:
						n[1]['cifname'] = v[1]
			
			for ea in eas:
	
				g += 1
	
				print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
				print('edge assignment : ',ea)
				print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
				print()
				
				type_assign = dict((k,[]) for k in sorted(TET, reverse=True))
				for k,m in zip(TET,ea):
					type_assign[k] = m
				
				# add cifname to TG.edge
				for e in TG.edges(data=True):
					ty = e[2]['type']
					for k in type_assign:
						if ty == k or (ty[1],ty[0]) == k:
							e[2]['cifname'] = type_assign[k]

				num_possible_XX_bonds = 0
				for edge_type, cifname in zip(TET, ea):
					if cifname == 'ntn_edge.cif':
						factor = 1
					else:
						factor = 2
					edge_type_count = edge_counts[edge_type]
					num_possible_XX_bonds += factor * edge_type_count

				ea_dict = assign_node_vecs2edges(TG, unit_cell, SYMMETRY_TOL, template)
				all_SBU_coords = SBU_coords(TG, ea_dict, CONNECTION_SITE_BOND_LENGTH)
				sc_a, sc_b, sc_c, sc_alpha, sc_beta, sc_gamma, sc_covar, Bstar_inv, max_length, callbackresults, ncra, ncca, scaling_data = scale(all_SBU_coords,a,b,c,ang_alpha,ang_beta,ang_gamma,max_le,num_vertices,Bstar,alpha,num_edges,FIX_UC,SCALING_ITERATIONS,PRE_SCALE,MIN_CELL_LENGTH,OPT_METHOD)
		
				print('*******************************************')
				print('The scaled unit cell parameters are : ')
				print('*******************************************')
				print('a    :', np.round(sc_a, 5))
				print('b    :', np.round(sc_b, 5))
				print('c    :', np.round(sc_c, 5))
				print('alpha:', np.round(sc_alpha, 5))
				print('beta :', np.round(sc_beta, 5))
				print('gamma:', np.round(sc_gamma, 5))
				print()
	
				for sc, name in zip((sc_a, sc_b, sc_c), ('a', 'b', 'c')):
					cflag = False
					if sc == MIN_CELL_LENGTH:
						print('unit cell parameter', name, 'may have collapsed during scaling!')
						print('try re-running with', name, 'fixed or a larger MIN_CELL_LENGTH')
						print('no cif will be written')
						cflag = True
	
				if cflag:
					continue
	
				scaled_params = [sc_a,sc_b,sc_c,sc_alpha,sc_beta,sc_gamma]
			
				sc_Alpha = np.r_[alpha[0:num_edges-num_vertices+1,:], sc_covar]
				sc_omega_plus = np.dot(Bstar_inv, sc_Alpha)
			
				ax = sc_a
				ay = 0.0
				az = 0.0
				bx = sc_b * np.cos(sc_gamma * pi/180.0)
				by = sc_b * np.sin(sc_gamma * pi/180.0)
				bz = 0.0
				cx = sc_c * np.cos(sc_beta * pi/180.0)
				cy = (sc_c * sc_b * np.cos(sc_alpha * pi/180.0) - bx * cx) / by
				cz = (sc_c ** 2.0 - cx ** 2.0 - cy ** 2.0) ** 0.5
				sc_unit_cell = np.asarray([[ax,ay,az],[bx,by,bz],[cx,cy,cz]]).T
				
				scaled_coords = omega2coords(start, TG, sc_omega_plus, (sc_a,sc_b,sc_c,sc_alpha,sc_beta,sc_gamma), num_vertices, template, g, WRITE_CHECK_FILES)
				nvecs,evecs,node_placed_edges = scaled_node_and_edge_vectors(scaled_coords, sc_omega_plus, sc_unit_cell, ea_dict)
				placed_nodes, node_bonds = place_nodes(nvecs, CHARGES, ORIENTATION_DEPENDENT_NODES)
				placed_edges, edge_bonds = place_edges(evecs, CHARGES, len(placed_nodes))
				#print(f"place_edges{place_edges}")
	
				if RECORD_CALLBACK:
	
					vnames = '_'.join([v.split('.')[0] for v in v_set])
	
					if len(ea) <= 5:
						enames = '_'.join([e[0:-4] for e in ea])
					else:
						enames = str(len(ea)) + '_edges'
	
					prefix = template[0:-4] + '_' +  vnames + '_' + enames
	
					frames = scaling_callback_animation(callbackresults, alpha, Bstar_inv, ncra, ncca, num_vertices, num_edges, TG, template, g, False)
					write_scaling_callback_animation(frames, prefix)
					animate_objective_minimization(callbackresults, prefix)
	
				if PLACE_EDGES_BETWEEN_CONNECTION_POINTS:
					placed_edges,cleaved_placed_edges,cleaved_placed_nodes,X_Opair = adjust_edges(placed_edges, placed_nodes, sc_unit_cell)
				
				# add classifination 
				#cleaved_placed_edges = placed_OXedges
				cleaved_placed_nodes = np.c_[cleaved_placed_nodes, np.array(['NODE' for i in range(len(cleaved_placed_nodes))])]
				cleaved_placed_edges = np.c_[cleaved_placed_edges, np.array(['EDGE' for i in range(len(cleaved_placed_edges))])]
				placed_nodes = np.c_[placed_nodes, np.array(['node' for i in range(len(placed_nodes))])]
				placed_edges = np.c_[placed_edges, np.array(['edge' for i in range(len(placed_edges))])]
				

				placed_all = list(placed_nodes) + list(placed_edges)
				bonds_all = node_bonds + edge_bonds
		
				if WRITE_CHECK_FILES:
					write_check_cif(template, placed_nodes, placed_edges, g, scaled_params, sc_unit_cell)
			
				if REMOVE_DUMMY_ATOMS:
					placed_all, bonds_all, nconnections = remove_Fr(placed_all,bonds_all)
				
				print('computing X-X bonds...')
				print()
				print('*******************************************')
				print('Bond formation : ')
				print('*******************************************')
				
				fixed_bonds, nbcount, bond_check_passed = bond_connected_components(placed_all, bonds_all, sc_unit_cell, max_length, BOND_TOL, nconnections, num_possible_XX_bonds)
				print('there were ', nbcount, ' X-X bonds formed')
					
				if bond_check_passed:
					print('bond check passed')
					bond_check_code = ''
				else:
					print('bond check failed, attempting distance search bonding...')
					fixed_bonds, nbcount = distance_search_bond(placed_all, bonds_all, sc_unit_cell, 2.5)
					bond_check_code = '_BOND_CHECK_FAILED'
					print('there were', nbcount, 'X-X bonds formed')
				print()
		
			#	if CHARGES:
			#		fc_placed_all, netcharge, onetcharge, rcb = fix_charges(placed_all)
			#	else:
			#		fc_placed_all = placed_all
			#
				fc_placed_all = placed_all
				fixed_bonds = fix_bond_sym(fixed_bonds, placed_all, sc_unit_cell)
	
			#	if CHARGES:
			#		print('*******************************************')
			#		print('Charge information :                       ')
			#		print('*******************************************')
			#		print('old net charge                  :', np.round(onetcharge, 5))
			#		print('rescaling magnitude             :', np.round(rcb, 5))
			#
			#		remove_net = choice(range(len(fc_placed_all)))
			#		fc_placed_all[remove_net][4] -= np.round(netcharge, 4)
			#
			#		print('new net charge (after rescaling):', np.sum([li[4] for li in fc_placed_all]))
			#		print()

				vnames = '_'.join([v.split('.')[0] for v in v_set])
				enames_list = [e[0:-4] for e in ea]
				enames_grouped = [list(edge_gr) for ind,edge_gr in itertools.groupby(enames_list)]
				enames_grouped = [(len(edge_gr), list(set(edge_gr))) for edge_gr in enames_grouped]
				enames_flat = [str(L) + '-' + '_'.join(names) for L,names in enames_grouped]
				enames = '_'.join(enames_flat)
				
			#	if catenation:
			#		cifname = template[0:-4] + '_' +  vnames + '_' + enames + bond_check_code + '_' + 'CAT' + str(cat_count) + '.cif'
			#	else:
			#		cifname = template[0:-4] + '_' +  vnames + '_' + enames + bond_check_code + '.cif'
		
				if WRITE_CIF:
					print('writing cif...')
					print()
					if len(cifname) > 255:
						cifname = cifname[0:241]+'_truncated.cif'
					write_cif(fc_placed_all, fixed_bonds, scaled_params, sc_unit_cell, cifname, CHARGES, wrap_coords=False)

if catenation and MERGE_CATENATED_NETS:
	
	print('merging catenated cifs...')
	cat_cifs = glob.glob('output_cifs/*_CAT*.cif')

	for comb in itertools.combinations(cat_cifs, cat_count):

		builds = [name[0:-9] for name in comb]

		print(set(builds))

		if len(set(builds)) == 1:
			pass
		else:
			continue

		merge_catenated_cifs(comb, CHARGES)

	#for cif in cat_cifs:
	#	os.remove(cif)


template : cds.cif

Number of vertices =  8
Number of edges =  16

*****************************************************************
RMSD of the compatible node BBs with assigned vertices:          
*****************************************************************

vertex V (4 connected)
     4c_Cu_1_Ch.cif deviation = 0.0 (within tolerance)
* 1 compatible building blocks out of 1 available for node V *

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
vertex assignment :  ['v1-4c_Cu_1_Ch.cif']
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
edge assignment :  ('1B_2F_Ch.cif',)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

scaling unit cell and vertex positions...
optimizing with local minimization algorithm L-BFGS-B...

The final objective function value is

<class 'networkx.utils.decorators.argmap'> compilation 30:3: FutureWarning: 

shortest_path will return an iterator that yields
(node, path) pairs instead of a dictionary when source
and target are unspecified beginning in version 3.5

To keep the current behavior, use:

	dict(nx.shortest_path(G))


In [36]:
def placed_arr(placed_all):
    placed_all_arr=np.empty((len(placed_all),len(placed_all[0])),dtype=object)
    for i in range(len(placed_all)):
            line = placed_all[i]
            placed_all_arr[i]=line

    placed_all_arr[:,1:4]=placed_all_arr[:,1:4].astype(float)
    placed_all_arr[:,6]=placed_all_arr[:,6].astype(int)
    res_id=np.unique(placed_all_arr[:,6]).astype(int)
    return placed_all_arr,res_id

def fetch_node_withidx(placed_node,idx_list):
    res_id_list = [i+1 for i in idx_list ]
    res=[]
    for n in res_id_list:
        res.append(placed_node[placed_node[:,6]==n])
    return np.vstack(res)

    
def fetch_edge_withidx(placed_edge,idx_list):
    res_id_list = [-1*i-1 for i in idx_list ]
    res=[]
    for n in res_id_list:
        res.append(placed_edge[placed_edge[:,6]==n])
    return np.vstack(res)



In [19]:
placed_nodes_arr,nodes_id=placed_arr(cleaved_placed_nodes)
placed_edges_arr,edges_id=placed_arr(cleaved_placed_edges)

nodes_idx =np.arange(1,1+len(TG.nodes()),1).tolist()
edges_idx =np.arange(1,1+len(TG.edges()),1).tolist()
t_nodes_idx = [1,2,3,4]#nodes_idx#[3,5,7,8]
t_edges_idx = [1,2,3,4]#edges_idx
target_nodes = fetch_node_withidx(placed_nodes_arr,t_nodes_idx)
target_edges = fetch_edge_withidx(placed_edges_arr,t_edges_idx)
target_all = np.vstack((target_nodes,target_edges))


In [20]:
def temp_xyz(output,placed_all):
    atoms_number = len(placed_all)
    newxyz = []
    with open( output, "w") as fp:
        newxyz.append(str(atoms_number) + "\n" + "generated by MOF_BUILD" + "\n")
        for i in range(atoms_number):

            value_label = placed_all[i][0]   # atom_label
            value_label = re.sub(r"\d", "", value_label)
            value_x = float(placed_all[i][1])  # x
            value_y = float(placed_all[i][2])   # y
            value_z = float(placed_all[i][3])   # z
            formatted_line = "%-5s%8.3f%8.3f%8.3f" % (
                value_label,
                value_x,
                value_y,
                value_z,
            )
            newxyz.append(formatted_line + "\n")
        fp.writelines(newxyz)

txyz="output_cifs/06tt.xyz"
temp_xyz(txyz,target_all)

In [33]:
def fc_assign_res_idx(placed_all):
    placed_all_arr=np.empty((len(placed_all),len(placed_all[0])),dtype=object)
    for i in range(len(placed_all)):
        line = placed_all[i]
        placed_all_arr[i]=line

    placed_all_arr[:,1:4]=placed_all_arr[:,1:4].astype(float)
    placed_all_arr[:,6]=placed_all_arr[:,6].astype(int)
    res_id=list(np.unique(placed_all_arr[:,6]).astype(int))
    
    res_id.sort(reverse=True)
    res_list=[]
    for i in range(len(res_id)):
        r_id = res_id[i]
        res_arr= placed_all_arr[placed_all_arr[:,6]==r_id]
        vec = res_arr[:,1:4]
        #cvec = np.dot(np.linalg.inv(sc_unit_cell),vec.T).T
        cvec = np.dot(vec,np.linalg.inv(sc_unit_cell))
        #if np.any(cvec<0):
        #    moded_cvec = np.mod(cvec, 1)
        #    row_diff = np.any(cvec != moded_cvec, axis=1)
        #    num_diff_rows = np.sum(row_diff)
        #    print(f"num_diff_rows{num_diff_rows},len(cvec){len(cvec)},new_res_id: {i}")
        #    #if 0<num_diff_rows: #< 0.5*len(cvec):
        #    one_diff_ind = np.where(row_diff)[0][0]
        #    #if np.any(cvec<0) or num_diff_rows>0.5*len(cvec):
        #    diff = moded_cvec[one_diff_ind]-cvec[one_diff_ind]
        #    print(f"diff{diff}")
        #    cvec[:, :3] += diff
        #    print(f"wrap this residue res_id: {r_id}, new_res_id: {i}")
        #        #cvec=moded_cvec
        
        res_arr[:,1:4] = cvec
        res_arr[:,6] = i+1
        res_list.append(res_arr)

    return np.vstack((res_list))

def outxyz(output,all_array):

    atoms_number = len(all_array)
    newxyz = []
    with open(output, "w") as fp:
        newxyz.append(str(atoms_number) + "\n" + "generated by MOF_BUILD" + "\n")
        for i in range(atoms_number):
            row = all_array[i]
            value_label = row[0]   # atom_label
            value_label = re.sub(r"\d", "", value_label)
            value_x = sc_a*float(row[1])  # x
            value_y = sc_b*float(row[2])  # y
            value_z = sc_c*float(row[3])  # z
            formatted_line = "%-5s%8.3f%8.3f%8.3f" % (
                value_label,
                value_x,
                value_y,
                value_z,
            )
            newxyz.append(formatted_line + "\n")
        fp.writelines(newxyz)

def outpdb(output,all_array):

        atoms_number = len(all_array)

        newpdb = []
        newpdb.append("generated by MOF_BUILD" + "\n")
        newpdb.append(str(atoms_number) + "\n")
        with open(output, "w") as fp:
            # Iterate over each line in the input file
            for i in range(atoms_number):
                line = all_array[i]
                # Extract values based on their positions in the format string
                value1 = "ATOM"
                value2 = int(i + 1)
                value3 = line[5]
                value4 = str(line[7])[0]  # residue
                value5 = line[6]  # residue number
                value6 = sc_a*float(line[1])  # x
                value7 = sc_b*float(line[2])  # y
                value8 = sc_c*float(line[3])  # z
                value9 = "1.00"
                value10 = "0.00"
                value11 = re.sub(r"\d", "", line[0]) # note
                # Format the values using the specified format string
                formatted_line = "%-6s%5d%5s%4s%10d%8.3f%8.3f%8.3f%6s%6s%4s" % (
                    value1,
                    value2,
                    value3,
                    value4,
                    value5,
                    value6,
                    value7,
                    value8,
                    value9,
                    value10,
                    value11,
                )
                newpdb.append(formatted_line + "\n")
            fp.writelines(newpdb)

def outgro(output,all_array):
        atoms_number = len(all_array)

        newgro = []
        with open(output, "w") as fp:
            newgro.append("generated by MOF_BUILD" + "\n" + str(atoms_number) + "\n")
            for i in range(atoms_number):
                line = all_array[i]

                value_atom_number = int(i + 1)  # atom_number
                value_label = line[0]  # atom_label
                value_resname = str(line[7])[0]  # residue_name
                value_resnumber = line[6] # residue number
                value_x = 0.1*sc_a*float(line[1])  # x
                value_y = 0.1*sc_b*float(line[2])  # y
                value_z = 0.1*sc_c*float(line[3])  # z
                formatted_line = "%5d%-5s%5s%5d%8.3f%8.3f%8.3f" % (
                    value_resnumber,
                    value_resname,
                    value_label,
                    value_atom_number,
                    value_x,
                    value_y,
                    value_z,
                )
                newgro.append(formatted_line + "\n")
            tail = "5 5 5 \n"
            newgro.append(tail)
            fp.writelines(newgro)


In [34]:

target_wrapped_array = fc_assign_res_idx(target_all)
xyz='output_cifs/03te.xyz'
pdb='output_cifs/04te.pdb'
gro='output_cifs/07te.gro'

#outxyz(xyz,all_array)
#outpdb(pdb,all_array)
outgro(gro,target_wrapped_array)

In [ ]:
#def _place_nodes(nvecs):

import numpy as np
from numpy.linalg import norm
from bbcif_properties import bb2array, X_vecs, bbbonds, bbcharges
from place_bbs import superimpose

import itertools
import re

placed_nbb_coords = []
placed_nbb_coords_extend = placed_nbb_coords.extend
all_bonds = []
all_bonds_extend = all_bonds.extend
ind_seg = 0
bbind = 1

for n in nvecs:
    bbind = bbind + 1
    name,cvec,cif,nvec = n
    ll = 0

    for v in nvec:
        mag = np.linalg.norm(v - np.average(nvec, axis = 0))
        if mag > ll:
            ll = mag

    bbxvec = np.array(X_vecs(cif,'nodes',False))

    #if ORIENTATION_DEPENDENT_NODES:
    nbbxvec = bbxvec
    #else:
    #	nbbxvec = np.array([ll*(v / np.linalg.norm(v)) for v in bbxvec])

    min_dist,rot,tran = superimpose(nbbxvec,nvec)

    all_bb = bb2array(cif, 'nodes') # extract node cif atoms as all_bb (building block)
    all_coords = np.array([v[1] for v in all_bb]) #node cif: atoms coordinate
    all_inds = np.array([v[0] for v in all_bb]) #node cif: atom identifier
    chg, elem = bbcharges(cif, 'nodes')  ##node cif: atom charge and atom element 
    all_names = [o + re.sub('[A-Za-z]','',p) for o,p in zip(elem,all_inds)] #all_names: element + numeric part in identifier relace X to C
    print(f"ind_seg{ind_seg}")
    
    '''!!!'''
    #may change ind_seg but can use residue count for every node, each loop is a complete node 
    all_names_indices = np.array([int(re.sub('[A-Za-z]','',e)) for e in all_names]) + ind_seg

    elem_dict = dict((k,'') for k in all_inds)
    for i,j in zip(all_inds, elem):
        elem_dict[i] = j

    ind_dict = dict((k,'') for k in all_inds)
    for i,j in zip(all_inds, all_names_indices):
        ind_dict[i] = j

    '''!!!'''
    #may use bonds to add dummy atoms , filter 'S' metal-Oxygen bond
    bonds = bbbonds(cif, 'nodes')

    anf = [str(elem_dict[n]) + str(ind_dict[n]) for n in all_inds]

    abf = []
    for b in bonds:
        b1 = str(elem_dict[b[0]]) + str(ind_dict[b[0]])
        b2 = str(elem_dict[b[1]]) + str(ind_dict[b[1]])
        abf.append([b1,b2] + b[2:])
    
    '''!!!'''
    aff_all = np.dot(all_coords,rot) + cvec #rotate and translate cif_node to correswponding node in G
    
    laff_all = np.c_[anf, aff_all, chg, all_inds, [bbind] * len(anf)]

    placed_nbb_coords_extend(laff_all)
    all_bonds_extend(abf)
    ind_seg = ind_seg + len(all_names)

	#return placed_nbb_coords, all_bonds